In [7]:
import pandas as pd
import numpy as np
from numpy.core.fromnumeric import nonzero


def is_pure(s):
    return len(set(s)) == 1

def most_common(a):
    (values,counts) = np.unique(a,return_counts=True)
    ind=np.argmax(counts)
    return values[ind]

def print_tree(d, depth = 0):
    for key, value in d.items():
        for i in range(depth):
                print(' ', end='')
        if type(value) is dict:
            #print(key, end=':\n')
            print_tree(value, depth + 1)
        else:
            print(key, end=': ')
            print(value)

def partition(s):
  return {i: (s==i).nonzero()[0]  for i in np.unique(s)}


def entropy(s):
  unique_,counts_ = np.unique(s,return_counts= True)
  freq_ = counts_.astype(float)/len(s)
  res = 0
  for i in freq_:
    res -= i * np.log2(i)
  return res

def mutual_information(x,y):
  entropy_set = entropy(y)
  unique,counts = np.unique(x ,return_counts= True)
  freq = counts.astype(float)/len(x)

  entropy_subset = 0
  for i,j in zip(freq,unique):
   entropy_subset += i * entropy(y[x == j ])

  return entropy_set - entropy_subset

def recursive_split(x, y):

  if is_pure(y) or len(y) == 0:
    return most_common(y)
  
  gain = np.array([mutual_information(feature,y) for feature in x.T ])
  
  print('features : ',features)
  print('Information Gain : ',gain)
  print('Best split is : ',features[np.argmax(gain)])
 
  x_ = np.argmax(gain)

  if np.all(gain < 1e-6):
    return most_common(y)

  sets = partition(x[:,x_])
  #print(sets)

  tree = {} 

  for k,v in sets.items():
    y_subset = y.take(v,axis = 0)
    x_subset = x.take(v,axis = 0)
    #print(x_subset,y_subset)
    tree["x_%d = %s" % (x_, k)] = recursive_split(x_subset, y_subset)

  return tree



In [3]:
df = pd.read_csv('InfoGain.csv')
df.head()

,price,airbag,capacity,maintenance,profitable
0,low,no,2,low,yes
1,low,yes,4,med,no
2,low,no,4,low,yes
3,low,no,4,high,no
4,med,no,4,med,no


In [8]:
df_ = df.copy()
df_ = df.drop('profitable',axis = 1)
x = np.array(df_).T

for i in range(len(x[2])):
  if x[2][i] == 2:
    x[2][i] = 'two'
  elif x[2][i] == 4:
    x[2][i] = 'four'
  elif x[2][i] == 5:
    x[2][i] = 'five'

x = x.T
x

array([['low', 'no', 'two', 'low'],
       ['low', 'yes', 'four', 'med'],
       ['low', 'no', 'four', 'low'],
       ['low', 'no', 'four', 'high'],
       ['med', 'no', 'four', 'med'],
       ['med', 'yes', 'four', 'med'],
       ['med', 'yes', 'two', 'high'],
       ['med', 'no', 'five', 'high'],
       ['high', 'yes', 'four', 'med'],
       ['high', 'yes', 'two', 'high'],
       ['high', 'yes', 'five', 'high']], dtype=object)

In [10]:
y = np.array(df['profitable'])
features = list(df_)
d = recursive_split(x,y)

features :  ['price', 'airbag', 'capacity', 'maintenance']
Information Gain :  [0.01631317 0.00723449 0.19813135 0.18905267]
Best split is :  capacity
features :  ['price', 'airbag', 'capacity', 'maintenance']
Information Gain :  [0.20751875 0.08170417 0.         0.33333333]
Best split is :  maintenance
features :  ['price', 'airbag', 'capacity', 'maintenance']
Information Gain :  [0.5        0.31127812 0.         0.        ]
Best split is :  price
features :  ['price', 'airbag', 'capacity', 'maintenance']
Information Gain :  [0. 1. 0. 0.]
Best split is :  airbag
features :  ['price', 'airbag', 'capacity', 'maintenance']
Information Gain :  [0.91829583 0.91829583 0.         0.91829583]
Best split is :  price


In [11]:
Tree = print_tree(d)
print(Tree)

x_2 = five: yes
 x_3 = high: no
 x_3 = low: yes
   x_0 = high: yes
  x_0 = low: no
     x_1 = no: no
   x_1 = yes: yes
 x_0 = high: no
 x_0 = low: yes
 x_0 = med: no
None
